# Data engineering

This assignment focuses on techniques for cleaning and transforming the data to handle challenges of missing, inaccurate, or incomplete data. Please refer to [Machine Learning productionization - Data engineering](#data-engineering) to learn more.

Fill `____` pieces of the below implementation in order to pass the assertions.


<!-- 
# removing rows with missing values from dataset: pandas.DataFrame dataset.dropna(inplace=True) # removing columns with ratio of missing values greater than a threshold dataset = dataset[dataset.columns[dataset.isnull().mean() <= THRESHOLD]]

Imputation. If the number of missing values is not small enough to be removed and not large enough to be a substantial proportion of the variable entries, you can replace the missing values in a numerical variable with the mean/median of the non-missing entries and the missing values in a categorical variable with the mode, which is the most frequent entry of the variable.

# imputing missing values in each column with the mean of the corresponding # column using scikit-learn from sklearn.impute import SimpleImputer imputer = SimpleImputer(strategy='mean') # transform the dataset imputed_dataset = imputer.fit_transform(dataset)

Identifying the group of attributes that uniquely identify a single record is very important and needs validation from a domain expert. Removing duplicates on this group leaves you with distinct records necessary for model training. This group acts as a key to performing multiple aggregate and transformations operations on the dataset like calculating rolling mean, backfilling null values, missing value imputation (details on this in next point), etc.
True duplicates, i.e., instances of the same data point, are usually removed. In this way, the increase of the sample weight on these points is eliminated, and the risk of any artificial inflation in the performance metrics is reduced -->

## Exploring dataset

> **Learning goal**: By the end of this subsection, you should be comfortable finding general information about the data stored in pandas DataFrames.

In order to explore this functionality, we will import the modefined version of Python scikit-learn library's iconic dataset **Iris**.

In [1]:
import pandas as pd
from sklearn.datasets import load_iris
import math

iris_df = pd.read_csv('../../../data/modefined_sklearn_iris_dataset.csv', index_col=0)
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2000
1,5.1,3.5,1.4,0.2000
2,4.7,3.2,1.3,0.2000
3,4.6,3.1,1.5,NaN
4,5.0,3.6,1.4,0.1000
...,...,...,...,...
145,6.7,3.0,5.2,2.3000
146,6.3,2.5,5.0,1.9000
147,6.5,3.0,5.2,2.0223
148,6.2,3.4,5.4,2.3000


To start off, print the summary of a DataFrame.

In [2]:
iris_df.shape

(150, 4)

```{quizdown}

## How many entries the Iris dataset has?

> Please refer to the output of above cell.  

- [ ] 50
- [ ] 100
- [x] 150
- [ ] 200

```

Next, let's check the actual content of the `DataFrame`.

In [3]:
# displying first 5 rows of our iris_df
iris_df.head(5)

# in the first five rows, which one's spepal length is 5.0cm?
assert iris_df.iloc[4, 0] == 5.0

Conversely, we can check the last few rows of the DataFrame.

In [4]:
# displying last 5 rows of our `iris_df`.
iris_df.tail(5)

# in the last five rows, which one's spepal width is 2.5cm?
assert iris_df.iloc[-4, 1] == 2.5

**Takeaway**: Even just by looking at the metadata about the information in a DataFrame or the first and last few values in one, you can get an immediate idea about the size, shape, and content of the data you are dealing with.

## Dealing with missing data


Missing data can cause inaccuracies as well as weak or biased results. Sometimes these can be resolved by a "reload" of the data, filling in the missing values with computation and code like Python, or simply just removing the value and corresponding data. There are numerous reasons for why data may be missing and the actions that are taken to resolve these missing values can be dependent on how and why they went missing in the first place.

> **Learning goal**: By the end of this subsection, you should know how to replace or remove null values from DataFrames.


In pandas, the `isnull()` and `notnull()` methods are your primary methods for detecting null data. Both return Boolean masks over your data. We will be using numpy for NaN values:

In [5]:
iris_isnull_df = iris_df.isnull()

print(iris_isnull_df)

# find one row with missing value
assert iris_isnull_df.iloc[3, 3] == True
assert math.isnan(iris_df.iloc[3, 3]) == True

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                False             False              False             False
1                False             False              False             False
2                False             False              False             False
3                False             False              False              True
4                False             False              False             False
..                 ...               ...                ...               ...
145              False             False              False             False
146              False             False              False             False
147              False             False              False             False
148              False             False              False             False
149              False             False              False             False

[150 rows x 4 columns]


In [6]:
# get all the rows with missing data
iris_with_missing_value_df = iris_df[iris_isnull_df.any(axis=1)] #axis=1 表示跨列执行即有一列为NAN即输出   

#loc基于标签和布尔数组
#iloc基于位置

assert iris_with_missing_value_df.shape[0] == 16

**Dropping null values**: Beyond identifying missing values, pandas provides a convenient means `dropna` to remove null values from Series and DataFrames. (Particularly on large data sets, it is often more advisable to simply remove missing [NA] values from your analysis than deal with them in other ways.) 

In [7]:
# remove all the rows with missing values
iris_with_dropna_on_row_df = iris_df.dropna(how='any')

assert iris_with_dropna_on_row_df.shape[0] == 134

In [8]:
# remove all the columns with missing values
iris_with_dropna_on_column_df = iris_df.dropna(axis=1,how='any') #跨列执行，故有NAN列被删除

assert iris_with_dropna_on_column_df.columns.shape[0] == 0

In [9]:
# remove all the rows with 2 (2) missing values
iris_with_dropna_2_values_on_rows_df = iris_df.dropna(thresh=2)

assert iris_with_dropna_2_values_on_rows_df.shape[0] == 144

# remove all the rows with 1 (3) missing values
iris_with_dropna_1_values_on_rows_df = iris_df.dropna(thresh=1)

assert iris_with_dropna_1_values_on_rows_df.shape[0] == 147

**Filling null values**: Depending on your dataset, it can sometimes make more sense to fill null values with valid ones rather than drop them. You could use `isnull` to do this in place, but that can be laborious, particularly if you have a lot of values to fill. Because this is such a common task in data science, pandas provides `fillna`, which returns a copy of the Series or DataFrame with the missing values replaced with one of your choosing. 

In [10]:
# fll all the missing values with 0
iris_with_fillna_df = iris_df.fillna(value=0)

# get all the rows with missing data
iris_with_missing_value_after_fillna_df = iris_with_fillna_df.isnull().loc[iris_with_fillna_df.isnull().any(axis=1)]

assert iris_with_missing_value_after_fillna_df.shape[0] == 0
# assert iris_with_fillna_df.iloc[, 3] == -1 没明白

In [11]:
# forward-fill null values, which is to use the last valid value to fill a null:
iris_with_fillna_forward_df =iris_df.fillna(method="ffill")

# get all the rows with missing data
iris_with_missing_value_after_fillna_forward_df = iris_with_fillna_forward_df.isnull().loc[iris_with_fillna_forward_df.isnull().any(axis=1)]

assert iris_with_missing_value_after_fillna_forward_df.shape[0] == 0
assert float(iris_with_fillna_forward_df.iloc[3, 3]) == 0.2

In [12]:
# back-fill null values, which is to use the next valid value to fill a null:
iris_with_fillna_back_df = iris_df.fillna(method="bfill")

# get all the rows with missing data
iris_with_missing_value_after_fillna_back_df = iris_with_fillna_back_df.isnull().loc[iris_with_fillna_forward_df.isnull().any(axis=1)]

assert iris_with_missing_value_after_fillna_back_df.shape[0] == 0
assert float(iris_with_fillna_back_df.iloc[3, 3]) == 0.1

## Removing duplicate data

Data that has more than one occurrence can produce inaccurate results and usually should be removed. This can be a common occurrence when joining two or more datasets together. However, there are instances where duplication in joined datasets contain pieces that can provide additional information and may need to be preserved.

> **Learning goal**: By the end of this subsection, you should be comfortable identifying and removing duplicate values from DataFrames.

In addition to missing data, you will often encounter duplicated data in real-world datasets. Fortunately, pandas provides an easy means of detecting and removing duplicate entries.

**Identifying duplicates**: You can easily spot duplicate values using the `duplicated`  method in pandas, which returns a Boolean mask indicating whether an entry in a DataFrame is a duplicate of an earlier one. Let's create another example DataFrame to see this in action.

In [13]:
iris_isduplicated_df = iris_df.duplicated()

print(iris_isduplicated_df)

# find one row with duplicated value
assert iris_isduplicated_df.iloc[1] == True

0      False
1       True
2      False
3      False
4      False
       ...  
145    False
146    False
147    False
148    False
149    False
Length: 150, dtype: bool


**Dropping duplicates**: `drop_duplicates` simply returns a copy of the data for which all of the duplicated values are False:

In [14]:
# remove all the rows with duplicated values
iris_with_drop_duplicates_on_df = iris_df.drop_duplicates()

assert iris_with_drop_duplicates_on_df.shape[0] == 143

Both `duplicated` and `drop_duplicates` default to consider all columns but you can specify that they examine only a subset of columns in your DataFrame:

In [15]:
# remove all the rows with duplicated values on column 'petal width (cm)'
iris_with_drop_duplicates_on_column_df = iris_df.drop_duplicates(subset=['petal width (cm)'])

assert iris_with_drop_duplicates_on_column_df.shape[0] == 27

## Handle inconsistent data

Depending on the source, data can have inconsistencies in how it’s presented. This can cause problems in searching for and representing the value, where it’s seen within the dataset but is not properly represented in visualizations or query results. Common formatting problems involve resolving whitespace, dates, and data types. Resolving formatting issues is typically up to the people who are using the data. For example, standards on how dates and numbers are presented can differ by country.

> **Learning goal**: By the end of this subsection, you should know how to handle the inconsistent data format in the DataFrame.

Let's cleaning up the **4th** column `petal width (cm)` to make sure there's no data entry inconsistencies in it. Firstly, we will use a convenient method `unique` from pandas to check the unique values of this column

In pandas, the `unique` method is a convenient way to unique values based on a hash table:

In [16]:
column_to_format = iris_df.iloc[:,3]
column_to_format_unique = column_to_format.unique()

print(column_to_format_unique)

# find one row with duplicated value
assert column_to_format_unique.shape[0] == 27

[0.2       nan 0.1    0.4    0.3    0.22   0.24   0.5    0.6    1.4
 1.5    1.3    1.6    1.     1.1    1.8    1.2    1.7    1.5012 2.5
 1.9    2.2    2.1    2.     2.4    2.3    2.0223]


Regardless the `nan` value, you may find the numeric valus are in different precision. More specifically, `1.` or `1.5012` are not in the same precision as other numbers. We want to append tailing `0` to numbers like `1.`, and round numbers like `1.5012` to `1.5`.

In [17]:
# firstly, let's apply `round`` to the values to make the precision all as .1f
formatted_column = column_to_format.round(decimals=1)

print(formatted_column.unique())

#assert formatted_column.unique().shape[0] == 23

[0.2 nan 0.1 0.4 0.3 0.5 0.6 1.4 1.5 1.3 1.6 1.  1.1 1.8 1.2 1.7 2.5 1.9
 2.2 2.1 2.  2.4 2.3]


In [18]:

# now, let's add tailing 0 if needed to make numbers like 1. to be 1.0. 
# You may need to filter the nan value while processing.
formatted_column = formatted_column.map('{:.1f}'.format,na_action='ignore')

print(formatted_column.unique())

assert formatted_column.unique().shape[0] == 23

['0.2' nan '0.1' '0.4' '0.3' '0.5' '0.6' '1.4' '1.5' '1.3' '1.6' '1.0'
 '1.1' '1.8' '1.2' '1.7' '2.5' '1.9' '2.2' '2.1' '2.0' '2.4' '2.3']


## At last

Let's apply all the methods above to make the data to be clean.

In [19]:
# remove all rows with missing values
no_missing_data_df = iris_df.dropna(how='any')

# remove all rows with duplicated values
no_missing_dup_data_df = no_missing_data_df.drop_duplicates()

# apply the precision .1f to all the numbers
cleand_df = no_missing_dup_data_df.round(decimals=1)

assert no_missing_data_df.shape[0] == 134
assert no_missing_dup_data_df.shape[0] == 129
assert cleand_df[cleand_df.columns[3]].unique().shape[0] == 22

Also, you could refer to below two for more about how to handle outlier.

- [Ways to Detect and Remove the Outliers](https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba)
- [Outlier!!! The Silent Killer](https://www.kaggle.com/code/nareshbhat/outlier-the-silent-killer)

## Acknowledgements

Thanks to Microsoft with creating the open source course [Data Science for Beginners](https://github.com/microsoft/Data-Science-For-Beginners). It contributes some of the content in this chapter.